In [1]:
from appgym.web_env import WebEnv

In [3]:
env = WebEnv("http://127.0.0.1:8080/", content_selector='.mobile-content')

In [4]:
state, actions = env.reset()

In [11]:
state.image.shape

(480, 300, 3)

In [5]:
len(actions)

360

In [16]:
_ = env.step(actions[320])

In [5]:
480 / 20

24.0

In [6]:
300 / 20

15.0

In [7]:
24*15

360

In [8]:
480 / 8

60.0

In [12]:
env._viewport()

Rect(x=250, y=60, width=300, height=480)

In [10]:
range(10)

range(0, 10)

In [12]:
list(range(1, 11))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [17]:
from appgym.agent import Agent

In [28]:
agent = Agent(actions)

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable

In [ ]:
s = Variable(torch.from_numpy(state.image).float())

In [43]:
s.shape

torch.Size([480, 300, 3])

In [45]:
s.view(3, 480, 300).unsqueeze(0).shape

torch.Size([1, 3, 480, 300])

In [49]:
img_state = s.view(3, 480, 300).unsqueeze(0)

In [50]:
img_state.shape

torch.Size([1, 3, 480, 300])

In [90]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200

In [120]:
class Model(nn.Module):

    def __init__(self, n_actions):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)
        self.out = nn.Linear(62016, n_actions)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.out(x.view(x.size(0), -1))
        return F.softmax(x, dim = 1)
#         return 


# res = m(img_state)

In [121]:
import random
import math

model = Model(len(actions))
steps_done = 0
def select_action(state, actions):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        print("Model")
        s = Variable(torch.from_numpy(state.image).float())
        img_state = s.view(3, 480, 300).unsqueeze(0)
        res = model(img_state)
        return res.multinomial().data
    else:
        print("Random")
        return torch.LongTensor([[random.randrange(len(actions))]])

In [123]:
# s = state

for _ in range(10):
    act = select_action(state, actions)
    act_idx = act[0][0]
    state, actions, reward = env.step(actions[act_idx])


Random
Random
Random
Random
Random
Model
Random
Random
Random
Random


In [115]:
act = select_action(state, actions)

Model


In [112]:
act_idx = act[0][0]

10

In [85]:
res.multinomial().data[0]


 246
[torch.LongTensor of size 1]

In [67]:
res.shape

torch.Size([1, 360])

In [59]:
res.view(res.size(0), -1).shape

torch.Size([1, 62016])